In [0]:
%sql
INSERT INTO wine.wine_quality
VALUES (7.4, 0.7, 0, 1.9, 0.076, 11, 0.9978, 3.51, 0.56, 9.4, Null, Null),
(7.4, 0.7, 0, 1.9, 0.076, 11, 0.9978, 3.51, 0.56, 9.4, NULL, 100)

num_affected_rows,num_inserted_rows
2,2


In [0]:
df = spark.sql("SELECT * FROM wine.wine_quality")
display(df.filter("quality is null"))

fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
7.4,0.7,0.0,1.9,0.076,11.0,0.9978,3.51,0.56,9.4,null,null
7.4,0.7,0.0,1.9,0.076,11.0,0.9978,3.51,0.56,9.4,100.0,null
7.4,0.7,0.0,1.9,0.076,11.0,0.9978,3.51,0.56,9.4,null,null


In [0]:
df = df.na.drop()
assert df.filter('quality is null').count() == 0

In [0]:
display(df.groupBy('quality').count().orderBy('count'))

quality,count
3,10
8,18
4,53
7,199
6,638
5,681


Databricks visualization. Run in Databricks to view.

In [0]:
aq = df.approxQuantile('quality',[0.25, 0.75], 0)
print(aq)

[5.0, 6.0]


In [0]:
iq = aq[1] - aq[0]

lowerb = aq[0] - (2 * iq)

upperb = aq[1] + (2 * iq)

print(lowerb)
print(upperb)

3.0
8.0


In [0]:
df = df.filter((df.quality < upperb )| (df.quality > lowerb))
display(df.orderBy('quality').head())

Row(fixed_acidity=11.6, volatile_acidity=0.58, citric_acid=0.66, residual_sugar=2.2, chlorides=0.074, free_sulfur_dioxide=10.0, total_sulfur_dioxide=47.0, density=1.0008, pH=3.25, sulphates=0.57, alcohol=9.0, quality=3)

In [0]:
df.write.saveAsTable('wine.clean_data')